In [1]:
%load_ext literary.module

# Configuration

In [1]:
import typing as tp
from functools import lru_cache
from pathlib import Path

from traitlets.config import (
    Config,
    ConfigFileNotFound,
    JSONFileConfigLoader,
    PyFileConfigLoader,
)

The Literary project uses a well-known file to configure the various components. We can implement the configuration in Python or JSON, so we search for any configuration file with the appropriate stem.

In [2]:
CONFIG_FILE_STEM = "literary_config"

To find the config file, we simply perform a breadth-first search of each given path, and return the first file with the appropriate stem.

In [3]:
@lru_cache()
def find_literary_config(path) -> Path:
    """Load the configuration for the current Literary project.

    :return:
    """
    # Look for any config file
    for p in path.glob(f"{CONFIG_FILE_STEM}.*"):
        return p

    # Visit parent
    if path.parents:
        return find_literary_config(path.parent)

    raise FileNotFoundError("Couldn't find config file")

Upon finding a configuration file, we can attempt to load it with the known loader classes.

In [4]:
@lru_cache()
def load_literary_config(path: Path) -> Config:
    """Load a project configuration file

    :param path: configuration file path
    :return:
    """
    for loader_cls in JSONFileConfigLoader, PyFileConfigLoader:
        loader = loader_cls(path.name, str(path.parent))
        try:
            return loader.load_config()
        except ConfigFileNotFound:
            continue

    raise ValueError(f"{path!r} was not a recognised config file")